In [4]:
%cd "C:\Git\BI0730"
import os
import sys
import duckdb
import re
sys.path.append('../../')
sys.path.append(rf"C:\Git\BI0730")
from libs.geral.utils import * 
from libs.denodo import conexaoDenodo as cnn
pd.options.display.float_format = '{:,.2f}'.format
pd.options.display.max_columns = None
pd.options.display.max_colwidth = None
user = os.getlogin()

In [5]:
# indicador_nova_principalidade_historico
query = '''SELECT * FROM indicador_nova_principalidade_historico'''
principalidade = cnn.baixar_consulta_manual(query)

In [ ]:

con = duckdb.connect()

path = rf"C:\Users\{user}\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\cia_pcp_indicador_principalidade_historico\\2025*.parquet"
query = rf'''
    CREATE TABLE dados AS
    SELECT * FROM read_parquet('{path}')
    
   
'''

con.execute(query)

In [ ]:

query = f''' 
      WITH tratados AS 
        (
            SELECT *,
            ROW_NUMBER() OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes DESC) AS rn,
            pontos_principalidade - LAG(pontos_principalidade)  OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes ASC) AS var_pontos

            FROM dados
            QUALIFY rn <= 4
            ORDER BY ano_Mes
        ),
        
        marcacoes AS (
            SELECT  
            *,
            CASE WHEN var_pontos < 0 THEN 1 ELSE 0 END AS queda_flag
            FROM tratados 
            WHERE rn <= 3
            
        ),
        checagem AS (
            SELECT *,
            SUM(queda_flag) OVER (PARTITION BY cpf_cnpj ORDER BY ano_mes) AS soma_quedas
            FROM marcacoes
        )
        
        SELECT  * FROM checagem WHERE soma_quedas >= 3 and ano_mes = (SELECT MAX(ano_mes) FROM checagem)
        --SELECT  * FROM checagem WHERE cpf_cnpj = '00362741948'
        

        '''
con.sql(query).to_df()
       

In [ ]:
saida = rf"C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira\associados_queda_principalidade.parquet"
tmp = saida + ".tmp"

# Executar a CTE (usa a variável 'query' já definida acima) e materializar
df_quedas = con.sql(query).fetchdf()

# Validar presença das colunas calculadas
esperadas_calc = ['var_pontos','queda_flag','soma_quedas']
faltan = [c for c in esperadas_calc if c not in df_quedas.columns]
if faltan:
    raise RuntimeError(f"Faltan columnas calculadas: {faltan}")

# Guardar atomicamente
df_quedas.to_parquet(tmp, index=False)
os.replace(tmp, saida)

In [ ]:
df = pd.read_parquet(rf"C:\\Users\\{USER}\\Sicredi\\TimeBI_0730 - Documentos\\01_Rotineiros\\33_GiroCarteira\\giro_de_carteira\\associados_queda_principalidade.parquet")
df.head()

In [ ]:
assodia =  carregar_associados_completo()

In [ ]:
assodia

In [ ]:
# Ajuste prévio de ano_atual para evitar NameError e definir marca de data (AAAAMM)
from datetime import date

# Garante que ANO exista
try:
    ANO
except NameError:
    ANO = date.today().year

ano_atual = ANO

# Marca de data para uso em nomes de arquivos (AAAAMM)
data = date.today().strftime('%Y%m')

In [ ]:
ANO = 2025
USER = os.getlogin()

def localizar_arquivo_principalidade(ano:int, base_parquet_dir:str,
                                     nome_base:str='cia_pcp_indicador_principalidade_historico'):
    """Retorna (caminho_escolhido, lista_arquivos_avaliados).
    Regras:
      1. Se existir diretório <base>/<nome_base>/ com vários parquets:
         - Considera arquivos *.parquet que contenham nome_base.
         - Extrai prefixo inicial AAAAMM antes do primeiro '_'. 
         - Primeiro tenta só os do ano solicitado (AAAAMM começa com str(ano)).
         - Se não houver para o ano, considera todos e pega o maior AAAAMM.
      2. Se não houver diretório, tenta arquivo único <base>/<nome_base>.parquet
    """
    dir_candidatos = os.path.join(base_parquet_dir, nome_base)
    avaliados = []
    selecionaveis = []
    if os.path.isdir(dir_candidatos):
        for f in os.listdir(dir_candidatos):
            if f.lower().endswith('.parquet') and nome_base in f:
                avaliados.append(f)
                m = re.match(r'^(\d{6})_', f)
                if m:
                    aaaamm = m.group(1)
                    try:
                        selecionaveis.append((int(aaaamm), f))
                    except ValueError:
                        pass
        # Filtrar por ano solicitado
        ano_prefix = str(ano)
        por_ano = [t for t in selecionaveis if str(t[0]).startswith(ano_prefix)]
        if por_ano:
            por_ano.sort()
            escolhido = por_ano[-1][1]
        elif selecionaveis:
            selecionaveis.sort()
            escolhido = selecionaveis[-1][1]
        else:
            # Nenhum com padrão AAAAMM_ => fallback arquivo único
            escolhido = None
        if escolhido:
            return os.path.join(dir_candidatos, escolhido), avaliados
        else:
            # fallback arquivo único na raiz
            return os.path.join(base_parquet_dir, f'{nome_base}.parquet'), avaliados
    else:
        # Diretório não existe => assumir arquivo único
        return os.path.join(base_parquet_dir, f'{nome_base}.parquet'), avaliados

principalidade_path, lista_avaliados = localizar_arquivo_principalidade(ANO, PATH_BASES_PARQUET)
print('[principalidade] candidatos avaliados:', lista_avaliados[:8], '...' if len(lista_avaliados) > 8 else '')
print('[principalidade] selecionado:', principalidade_path)

RUTAS = {
    'inadimplentes': os.path.join(PATH_BASES_PARQUET, 'base_inadimplentes.parquet'),
    'associados_dir': os.path.join(PATH_BASES_PARQUET, 'associados_total_diario'),
    'saida_dir': fr'C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira',
    'saida_dir_giro': fr'C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira',
    'principalidade': principalidade_path
}

os.makedirs(RUTAS['saida_dir_giro'], exist_ok=True)

config = {
    'ano': ano_atual,
    'pasta_saida_duckdb': rf"C:\\Users\\{USER}\\Sicredi\\TimeBI_0730 - Documentos\\01_Rotineiros\\33_GiroCarteira\\giro_de_carteira\\associados_queda_principalidade.parquet",
    'pasta_isa_historico': rf"C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira",
    'arquivo_saida': rf"C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira_2025.parquet",
    'filtros_risco_bbm': ["BAIXÍSSIMO", "BAIXO 1", "BAIXO 2", "MÉDIO 1", "MÉDIO 2"],
    'dias_sem_movimentacao': (20, 45)
}
PARAMS = {
    'filtros_risco_bbm': ["BAIXÍSSIMO", "BAIXO 1", "BAIXO 2", "MÉDIO 1", "MÉDIO 2"],
    'dias_sem_movimentacao': (20, 45),
    'principalidade': ['sow']
}

for k, v in RUTAS.items():
    print(f'{k}: {v}')

el archivo que sustenta es el archivo de principalidade, los merge serán realizados a este archivo para poder filtrar por associados sin inadimplencia y que no tengan uso en el campo 'mobi_transacionou_30d' ó ultimo movimento entre 20 - 45 días

serán generados 2 parquet uno con toda la información y un dataframe construido con la siguientes campos e informaciones

{  
  "titulo": "string",    
  "usuarioResponsavel": "string", ----> GESTOR
  "usuarioSolicitante": "string", -----> 
  "numeroAgencia": 0, -----> cod_agencia
  "descricao": "string",  --------> descricao texto com detalhes e indices do associado + o texto da oferta de produtos a entregar
  "statusChamado": "RASCUNHO", 
  "departamentoId": 0,
  "nomeDepartamento": "string", --------->  _nom_agencia
  "dataNecessidade": "2025-08-14T11:45:45.991Z",
  "categoriaChamado": {
    "categoriaChamadoId": 0,  ---- > 1 ou 2
    "nomeCategoria": "string", ------> 
    "descricaoCategoria": "string", 
    "dataCriacao": "2025-08-14",
    "departamentoId": 0}
}

Titulo () + nome associado + cpf 

Descrição:
PF OU PJ
ISA atual 
ISA 3M
MC TOTAL ATUAL
MC 6M
INDICES PRINCIPALIDADE

OFERTA : TEXTO conformado por  categoria + descriçao categoria + produto  1 texto : 

  Oferta Acesso  ----->  Categoria 1 
(descriçao categoria) Ofertar Produtos/Serviços Básicos : {} ou [] {add. Produto}-------> Titulo + nome associado + cpf  
. Chave PIX ativa 
· Débito automático
· Cartão de débito
· Cartão de crédito
. Canais digitais
. Credenciamento ativo
· Cobrança
. Folha de Pagamento
. Domicílio
. Open Finance (receptor)

 Oferta Fluxo de Caixa   ----->  Categoria 2 
if % produto in range 
(descriçao categoria) Ofertar Movimentações : [] ou {} {add. Crédito &| cartão}-------> Titulo + nome associado + cpf
. Somatório do total de movimentações:
- Receber (Cash-in)
- Pagar (Cash-out)

Crédito - cartão
. SOW (Share of Wallet)
referente à cartão

Credito - outros
SOW (Share of Wallet)
referente à créditos



df

el campo categoria puede tener dos contenidos acesso y fluxo de caixa será utilizado como una flag de la condicion de los productos que no tiene 

el campo producto deberá conter todos los productos que no tiene haciendo un array para luego ser utilizado en la construccion del texto del campo descripcion tendremos 2 campos productos categoria I y productos categoria II


la categoria fluxo de caixa tiene percentuais que deberán ser utilizados con rangos para saber si existe posibilidad de un upgrade, se mantiene o que producto ofertar, esto deberá ser gravado en el campo produto ahora colocaremos solo los porcentages que existem en el sow, indicando a que corresponden, para esso utilizaremos el titulo del campo e la cifra contenido en el valor del campo

al final de todo concatenaremos todos los textos de los campos 


Titulo () + nome associado + cpf 

Descrição:
PF OU PJ
ISA atual 
ISA 3M
MC TOTAL ATUAL
MC 6M
INDICES PRINCIPALIDADE

OFERTA : TEXTO conformado por  categoria + descriçao categoria + produto  1 texto 


y crearemos el campo Descripcion 


deberá ser generado un codigo que implemente filtros de indimplencia, risco bbm y que la ultima movimentacion dejó de entre 20 y 45
será construida el campo descripcion con las caracteristia de los datos existentes en los parquets utilizados según el modelo, si el associado NO tiene el producto será addicionado como producto en la descripción con el titulo del campo como catgroria 



las columnas para este último parquet o dataframe serán 
{  
  "titulo": "string",    
  "usuarioResponsavel": "string", ----> GESTOR
  "usuarioSolicitante": "string", -----> 
  "numeroAgencia": 0, -----> cod_agencia
  "descricao": "string",  --------> descricao texto com detalhes e indices do associado + o texto da oferta de produtos a entregar
  "statusChamado": "RASCUNHO", 
  "departamentoId": 0,
  "nomeDepartamento": "string", --------->  _nom_agencia
  "dataNecessidade": "2025-08-14T11:45:45.991Z", ----> exemplo
  "categoriaChamado": {
    "categoriaChamadoId": 0,  ---- > 1 ou 2
    "nomeCategoria": "string", ------> 
    "descricaoCategoria": "string", 
    "dataCriacao": "2025-08-14",
    "departamentoId": 0}
}


las bases son conformadas mensualmente por lo que utilizaremos una data base de mes pero dividremos el total en 4 partes, para que sea usado por semana, colocando los mas antiguos para ser contactados la primera semana y así sucesivamente hasta nuevamente ser generado un nuevo archivo para eso generaremos la informacion en el campo dataNecessidade



del dashboard associados total dashboard
utilizaremos cpf_cnpj e nome_agencia,mc_total_1,gestor
C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\associados_totais_tratados_dashboard.parquet

de isa historico

C:\Users\carola_luco\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira\isa_historico_analise_20250814.parquet utilizaremos 
gestor,
isa_status
isa_media
isa_202503
isa_202504
isa_202505
isa_202506  siempre utilizando los ultimos 3 

de acuerdo a todo lo dicho necesito que generes un ipynb de debug con las rutas y las condiciones generadas,  utilizaremos pandas numpy re os sys 

In [ ]:
# Colunas arquivo principalidade
colunas_principalidade = ['ano_mes', 'cpf_cnpj', 'segmento', 'cod_cooperativa', 'cod_central',
       'cod_agencia', 'cod_carteira', 'status_associado', 'porte_padrao',
       'meses_desde_associacao', 'assoc_desde', 'nivel_risco', 
       'vlr_capital_social',  'pix_trans_30d', 'cad_pix',
       'sum_titulo_investimento', 'mobi_transacionou_30d', 'sum_previdencia',
       'sum_investimentos', 'total_valor_cartao_mercado_scr',
       'total_valor_cartao_sicredi', 'total_valor_outros_sicredi',  'sow_cartao',
       'possui_cartao_credito', 'debito_conta_ativo', 'possui_domicilio',
       'possui_cartao_debito', 'possui_folha_pagamento', 'possui_cobranca',
       'possui_adquirencia', 'transacao_app', 'flg_ativo', 'flg_novo_assoc',  'cad_pix_ativo',
       'ativou_open_finance', 'cash_in', 'cash_out', 'cash_total_fator',
       'cash_total', 'pontos_produtos_basicos',  'flag_principalidade',
       'pontos_principalidade', 'faixa_atingimento_meta',
       'percentual_atingimento',  'faixa_categoria',   'var_pontos',
       'queda_flag', 'soma_quedas']
colunas_principalidade

In [ ]:
# Carregar parquet base gerado pela célula DuckDB e restringir ESTRITAMENTE às colunas definidas
import os, pandas as pd
ruta_base = rf"C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\01_Rotineiros\33_GiroCarteira\giro_de_carteira\associados_queda_principalidade.parquet"
print('Lendo parquet base (fonte única principalidade):', ruta_base)
if not os.path.exists(ruta_base):
    raise FileNotFoundError('Parquet de principalidade não encontrado. Execute antes a célula DuckDB que o gera.')
_tmp = pd.read_parquet(ruta_base)
# Validar presença integral das colunas obrigatórias
faltantes = [c for c in colunas_principalidade if c not in _tmp.columns]
if faltantes:
    raise ValueError(f'Colunas obrigatórias ausentes no parquet: {faltantes}')
# Manter somente as colunas definidas (ordem garantida)
base_princ = _tmp[colunas_principalidade].copy()
print('Linhas:', len(base_princ), '| Colunas mantidas:', len(base_princ.columns))
base_princ.head(3)

In [ ]:
# (Removido salvamento antigo redundante) Placeholder para evitar erro.
print('Utilize a célula de montagem final para salvar os parquets.')

In [ ]:
# Derivar produtos faltantes (executar após filtros e merges)
# Diferenciar PF vs PJ: assumir segmento ou comprimento do documento (11=PF, >11=PJ) se não houver coluna específica

prod_pf = {
    'Chave PIX ativa': 'cad_pix_ativo',
    'Débito automático': 'debito_conta_ativo',
    'Conta salário': 'possui_folha_pagamento',
    'Cartão de débito': 'possui_cartao_debito',
    'Cartão de crédito': 'possui_cartao_credito',
    'Canais digitais': 'transacao_app',
    'Open Finance (receptor)': 'ativou_open_finance'
}
prod_pj = {
    'Chave PIX ativa': 'cad_pix_ativo',
    'Débito automático': 'debito_conta_ativo',
    'Cartão de débito': 'possui_cartao_debito',
    'Cartão de crédito': 'possui_cartao_credito',
    'Canais digitais': 'transacao_app',
    'Open Finance (receptor)': 'ativou_open_finance',
    'Domicílio': 'possui_domicilio',
    'Credenciamento ativo': 'possui_adquirencia',
    'Cobrança': 'possui_cobranca',
    'Folha de Pagamento': 'possui_folha_pagamento'
}

# Métricas fluxo / SOW (iguais para ambos)
prod_cat2_metricas = {
    'Cash-in': 'cash_in',
    'Cash-out': 'cash_out',
    'Total movimentações': 'cash_total',
    'SOW Cartão': 'sow_cartao'
}

# Inferir tipo_pessoa
base_princ['tipo_pessoa'] = np.where(base_princ['cpf_cnpj'].str.len()==11, 'PF','PJ')

# Normalizar booleans
for col in set(list(prod_pf.values()) + list(prod_pj.values())):
    if col in base_princ.columns:
        base_princ[col] = base_princ[col].replace({'S':1,'N':0}).fillna(0).astype(float)

# Soma de chaves PIX fortes (se existisse colunas detalhadas, placeholder; usar cad_pix_ativo como 1 ou 0)
base_princ['qtde_chaves_pix_fortes'] = base_princ.get('cad_pix_ativo', pd.Series([0]*len(base_princ))).fillna(0)

# Produtos faltantes conforme tipo

def faltantes_prod(r):
    mapping = prod_pf if r['tipo_pessoa']=='PF' else prod_pj
    return [n for n,c in mapping.items() if c in base_princ.columns and r.get(c,0)==0]

base_princ['prod_cat1_faltantes'] = base_princ.apply(faltantes_prod, axis=1)

# Métricas categoria 2
base_princ['prod_cat2_metricas'] = base_princ.apply(
    lambda r: [f"{n}: {r.get(c)}" for n,c in prod_cat2_metricas.items() if c in base_princ.columns], axis=1)

base_princ.head(1)

In [ ]:
base_princ

In [ ]:
# Preparar y enriquecer la base principal: produtos, métricas, descripciones y salida
from datetime import datetime, timedelta
import os
import pandas as pd
import numpy as np

if 'base_princ' not in globals():
    raise RuntimeError('base_princ não carregada. Execute a célula de leitura do parquet base antes desta.')

# Mapeo de productos (clave descriptiva -> columna en base_princ)
prod_cols_map = {
    'Chave PIX ativa': 'cad_pix_ativo',
    'Débito automático': 'debito_conta_ativo',
    'Conta salário': 'possui_folha_pagamento',
    'Cartão de débito': 'possui_cartao_debito',
    'Cartão de crédito': 'possui_cartao_credito',
    'Canais digitais': 'transacao_app',
    'Open Finance (receptor)': 'ativou_open_finance',
    'Domicílio': 'possui_domicilio',
    'Credenciamento ativo': 'possui_adquirencia',
    'Cobrança': 'possui_cobranca'
}

# Normalizar columnas de productos (S/N -> 1/0) si existen
for col in set(prod_cols_map.values()):
    if col in base_princ.columns:
        base_princ[col] = base_princ[col].replace({'S': 1, 'N': 0}).fillna(0).astype(float)

# Inferir tipo_pessoa si no existe
if 'tipo_pessoa' not in base_princ.columns and 'cpf_cnpj' in base_princ.columns:
    base_princ['tipo_pessoa'] = np.where(base_princ['cpf_cnpj'].astype(str).str.len() == 11, 'PF', 'PJ')

# Función para listar productos faltantes por cliente
def _falt(r):
    tipo = r.get('tipo_pessoa', 'PF')
    # Para PF excluimos productos que no aplican típicamente
    excluidos_pf = {'Domicílio', 'Credenciamento ativo', 'Cobrança'}
    activos = {k: v for k, v in prod_cols_map.items() if not (tipo == 'PF' and k in excluidos_pf)}
    faltantes = []
    for nome, col in activos.items():
        if col in r.index and pd.notnull(r.get(col)):
            try:
                if float(r.get(col)) == 0:
                    faltantes.append(nome)
            except Exception:
                # si no es numérico, considerarlo faltante si es falsy
                if not r.get(col):
                    faltantes.append(nome)
        else:
            # si la columna no existe en el registro, lo ignoramos
            continue
    return faltantes

base_princ['prod_cat1_faltantes'] = base_princ.apply(_falt, axis=1)

# Métricas de fluxo / SOW: etiquetas -> columnas esperadas en la base
prod_cat2_metricas = {
    'Cash-in': 'cash_in',
    'Cash-out': 'cash_out',
    'Cash-total movimentações': 'cash_total',
    'SOW Total Cartão': 'sow_cartao'
}

def _fmt_val(v):
    try:
        return f"{float(v):.2f}"
    except Exception:
        return ''

def _monta_metricas(r):
    itens = []
    for nome, col in prod_cat2_metricas.items():
        if col in r.index and pd.notnull(r.get(col)):
            itens.append(f"{nome}: {_fmt_val(r.get(col))}")
    return itens

base_princ['prod_cat2_metricas'] = base_princ.apply(_monta_metricas, axis=1)

# Título para o chamado
base_princ['titulo'] = base_princ.apply(lambda r: f"Oferta Principalidade - {r.get('cpf_cnpj', '')}", axis=1)

# Determinar columna ISA atual
col_isa_atual = None
if 'isa_atual_col' in globals() and isinstance(isa_atual_col, str) and isa_atual_col in base_princ.columns:
    col_isa_atual = isa_atual_col
elif 'isa_media' in base_princ.columns:
    col_isa_atual = 'isa_media'

def _fmt_isa_media3m(v):
    try:
        return f"{float(v):.2f}"
    except Exception:
        return ''

if col_isa_atual:
    def _bloco_isa(r):
        atual = r.get(col_isa_atual, '')
        media3m = _fmt_isa_media3m(r.get('isa_media_3m_calc')) if 'isa_media_3m_calc' in r.index else ''
        partes = []
        if pd.notnull(atual) and atual != '':
            partes.append(f"ISA atual: {atual}")
        if media3m:
            partes.append(f"ISA média (3M): {media3m}")
        return ' | '.join(partes)
    base_princ['bloco_isa'] = base_princ.apply(_bloco_isa, axis=1)
else:
    base_princ['bloco_isa'] = ''

# Bloques de productos e indicadores
base_princ['bloco_cat1'] = base_princ['prod_cat1_faltantes'].apply(lambda lst: 'Produtos básicos completos' if not lst else 'Produtos a ofertar: ' + ', '.join(lst))
base_princ['bloco_cat2'] = base_princ['prod_cat2_metricas'].apply(lambda lst: ' / '.join(lst) if lst else '')

# Descripción combinada
def _descricao(r):
    parts = [f"Segmento: {r.get('segmento','')}", f"MC | Total: {r.get('cash_total','')}"]
    if r.get('bloco_isa'): parts.append(r.get('bloco_isa'))
    if r.get('bloco_cat1'): parts.append(r.get('bloco_cat1'))
    if r.get('bloco_cat2'): parts.append(r.get('bloco_cat2'))
    return ' | '.join([p for p in parts if p])

base_princ['descricao'] = base_princ.apply(_descricao, axis=1)

# Categoria do chamado
base_princ['categoriaChamadoId'] = base_princ['prod_cat1_faltantes'].apply(lambda l: 1 if len(l) > 0 else 2)
nome_map = {1: 'Acesso', 2: 'Fluxo de Caixa'}
base_princ['nomeCategoria'] = base_princ['categoriaChamadoId'].map(nome_map)

desc_map = {1: 'Ofertar Básicos', 2: 'Ofertar Serviços de Movimentações'}
base_princ['descricaoCategoria'] = base_princ['categoriaChamadoId'].map(desc_map)

# Ordenar por pontos de principalidade se existir
if 'pontos_principalidade' in base_princ.columns:
    base_princ = base_princ.sort_values('pontos_principalidade').reset_index(drop=True)
else:
    base_princ = base_princ.reset_index(drop=True)

# Distribuir datas de necessidade ciclando por 4 semanas (0,7,14,21)
qtd = len(base_princ)
ini = datetime.today().date()
semanas = [0, 7, 14, 21]
if qtd > 0:
    base_princ['dataNecessidade'] = base_princ.index.map(lambda i: (ini + timedelta(days=semanas[i % len(semanas)])).isoformat() + 'T08:00:00.000Z')
else:
    base_princ['dataNecessidade'] = []

# Montar campo categoriaChamado (estrutura esperada)
def _monta_categoria(r):
    return {
        'categoriaChamadoId': int(r['categoriaChamadoId']),
        'nomeCategoria': r.get('nomeCategoria',''),
        'descricaoCategoria': r.get('descricaoCategoria',''),
        'dataCriacao': datetime.today().date().isoformat(),
        'departamentoId': int(r.get('cod_agencia', 0))
    }

base_princ['categoriaChamado'] = base_princ.apply(_monta_categoria, axis=1)

# Preparar base_final renombrando columnas esperadas
base_final = base_princ.rename(columns={'cod_agencia': 'numeroAgencia', 'nome_agencia': 'nomeDepartamento'})

# Responsable / solicitante
usuario_default = globals().get('user', '') or globals().get('USER', '')
base_final['usuarioResponsavel'] = base_final.get('gestor', usuario_default)
base_final['usuarioSolicitante'] = base_final['usuarioResponsavel']
base_final['statusChamado'] = 'RASCUNHO'
base_final['departamentoId'] = base_final['numeroAgencia']

# Selección columnas de salida
cols_saida = ['titulo', 'usuarioResponsavel', 'usuarioSolicitante', 'numeroAgencia',
             'descricao', 'statusChamado', 'departamentoId', 'nomeDepartamento',
             'dataNecessidade', 'categoriaChamado']

resultado_chamados = base_final.loc[:, [c for c in cols_saida if c in base_final.columns]].copy()

# Salvar arquivos parquet
saida_dir = RUTAS.get('saida_dir_giro') if isinstance(RUTAS, dict) else None
if saida_dir:
    parquet_completo = os.path.join(saida_dir, 'giro_completo_principalidade.parquet')
    parquet_chamados = os.path.join(saida_dir, 'giro_chamados_principalidade.parquet')
    try:
        base_princ.to_parquet(parquet_completo, index=False)
        resultado_chamados.to_parquet(parquet_chamados, index=False)
        print({'completo': parquet_completo, 'chamados': parquet_chamados, 'linhas_chamados': len(resultado_chamados)})
    except Exception as e:
        print('Erro ao salvar parquet:', e)
else:
    print('RUTAS["saida_dir_giro"] não encontrada; não foi possível salvar os arquivos.')

resultado_chamados.to_parquet(rf"")


In [ ]:
# Aplicar filtros e preparar base principal (usa base_princ já carregada do parquet associados_queda_principalidade)
if 'base_princ' not in globals():
    raise RuntimeError('base_princ não carregada. Execute a célula de leitura do parquet base antes desta.')

if base_princ.empty:
    raise ValueError('base_princ está vazia. Verifique o parquet base.')

# Normalizar chave
def normaliza_doc(valor):
    if pd.isna(valor):
        return valor
    v = re.sub(r'\D','', str(valor))
    return v.lstrip('0')

if 'cpf_cnpj' not in base_princ.columns:
    possiveis_chaves = [c for c in base_princ.columns if 'cpf' in c.lower() or 'cnpj' in c.lower() or 'doc' in c.lower()]
    if possiveis_chaves:
        base_princ.rename(columns={possiveis_chaves[0]:'cpf_cnpj'}, inplace=True)
if 'cpf_cnpj' not in base_princ.columns:
    raise ValueError('Coluna cpf_cnpj ausente na base principalidade mês.')
base_princ['cpf_cnpj'] = base_princ['cpf_cnpj'].map(normaliza_doc)

# Inadimplentes
path_inad = RUTAS['inadimplentes']
if os.path.exists(path_inad):
    inad = pd.read_parquet(path_inad)
    for alt in ['cpf','cnpj','documento','doc','num_cpf_cnpj']:
        if alt in inad.columns and 'cpf_cnpj' not in inad.columns:
            inad.rename(columns={alt:'cpf_cnpj'}, inplace=True)
    if 'cpf_cnpj' not in inad.columns:
        inad['cpf_cnpj'] = inad.iloc[:,0]
    inad['cpf_cnpj'] = inad['cpf_cnpj'].map(normaliza_doc)
    inad = inad[['cpf_cnpj']].drop_duplicates()
    inad['flag_inadimplente'] = 1
    base_princ = base_princ.merge(inad, on='cpf_cnpj', how='left')
else:
    base_princ['flag_inadimplente'] = np.nan

# Risco BBM (normalizar caixa / acentuação mínima)
if 'nivel_risco' in base_princ.columns:
    base_princ['nivel_risco_norm'] = base_princ['nivel_risco'].astype(str).str.upper().str.strip()
    riscos_ok = [r.upper() for r in PARAMS['filtros_risco_bbm']]
    base_princ = base_princ[base_princ['nivel_risco_norm'].isin(riscos_ok)]
else:
    print('Aviso: coluna nivel_risco ausente.')

# Sem inadimplência
base_princ = base_princ[(base_princ['flag_inadimplente'].isna()) | (base_princ['flag_inadimplente']!=1)]

# Merge dashboard (inclui ult_movimento)
if 'assoc_dash' in globals():
    assoc_dash['cpf_cnpj'] = assoc_dash['cpf_cnpj'].map(normaliza_doc)
    base_princ = base_princ.merge(assoc_dash, on='cpf_cnpj', how='left')
else:
    print('Dashboard não carregado: execute a célula de carga do dashboard/ISA antes desta.')

# Filtro movimento 20-45 dias usando ult_movimento
if 'ult_movimento' in base_princ.columns:
    hoje = pd.Timestamp.today().normalize()
    base_princ['ult_movimento_dt'] = pd.to_datetime(base_princ['ult_movimento'], errors='coerce')
    base_princ['dias_sem_mov'] = (hoje - base_princ['ult_movimento_dt']).dt.days
    lim_inf, lim_sup = PARAMS['dias_sem_movimentacao']
    base_princ = base_princ[(base_princ['dias_sem_mov']>=lim_inf) & (base_princ['dias_sem_mov']<=lim_sup)]
else:
    print('Coluna ult_movimento ausente no dashboard para filtro de movimento.')

# Merge ISA se disponível
if 'isa_sel' in globals() and isinstance(isa_sel, pd.DataFrame) and not isa_sel.empty and 'cpf_cnpj' in isa_sel.columns:
    isa_sel['cpf_cnpj'] = isa_sel['cpf_cnpj'].map(normaliza_doc)
    if 'gestor' in base_princ.columns and 'gestor' in isa_sel.columns:
        base_princ = base_princ.merge(isa_sel, on=['cpf_cnpj','gestor'], how='left')
    else:
        base_princ = base_princ.merge(isa_sel, on='cpf_cnpj', how='left')
else:
    base_princ['isa_media_3m_calc'] = np.nan

# Corte ISA >= 2 (média atual ou média 3M). Remove quem está fora.
if 'isa_media' in base_princ.columns:
    base_princ = base_princ[(base_princ['isa_media'].fillna(0) >= 2) | (base_princ.get('isa_media_3m_calc', pd.Series(dtype=float)).fillna(0) >= 2)]

print('Base após filtros (inclui corte ISA>=2):', base_princ.shape)
base_princ.head(2)

In [ ]:
# Gerar parquets: completo (dados filtrados) e card Kanban
import json

if 'base_princ' not in globals() or 'resultado_chamados' not in globals():
    raise RuntimeError('Execute as células anteriores antes de gerar os parquets finais.')

# 1. Parquet completo já salvo anteriormente, mas reforçamos aqui se desejar atualizar
parquet_completo = os.path.join(RUTAS['saida_dir_giro'], 'giro_completo_principalidade.parquet')
base_princ.to_parquet(parquet_completo, index=False)

# 2. Parquet apenas com campos necessários para o card (Kanban)
cols_card = [
    'titulo',
    'descricao',
    'categoriaChamado',
    'usuarioResponsavel',
    'usuarioSolicitante',
    'dataNecessidade',
    'statusChamado',
    'numeroAgencia',
    'nomeDepartamento',
    'departamentoId'
]
card_df = resultado_chamados[cols_card].copy()

# Opcional: serializar categoriaChamado em JSON para consumo externo se necessário
card_df['categoriaChamado_json'] = card_df['categoriaChamado'].apply(lambda x: json.dumps(x, ensure_ascii=False))

parquet_card = os.path.join(RUTAS['saida_dir_giro'], 'giro_cards_kanban.parquet')
card_df.to_parquet(parquet_card, index=False)

print({'parquet_completo': parquet_completo, 'parquet_card': parquet_card, 'linhas_card': len(card_df)})
card_df.head(1)

In [ ]:
# PIPELINE FINAL UNIFICADA
import json
from datetime import datetime, timedelta

if 'base_princ' not in globals():
    raise RuntimeError('base_princ ausente. Execute leitura base.')

# Garantir campos essenciais derivados
if 'prod_cat1_faltantes' not in base_princ.columns:
    raise RuntimeError('Execute a célula de derivação de produtos antes da pipeline final.')

# Família pontuação por grupos (Acesso = produtos básicos, Fluxo = cash, SOW = sow_cartao)
base_princ['score_acesso'] = base_princ.get('pontos_produtos_basicos', np.nan)
base_princ['score_fluxo'] = base_princ.get('cash_total_fator', np.nan)
base_princ['score_sow'] = base_princ.get('sow_cartao', np.nan)

# Classificação novo associado (flg_novo_assoc ou meses_desde_associacao)
base_princ['class_assoc'] = np.where(
    (base_princ.get('flg_novo_assoc',0)==1) | (base_princ.get('meses_desde_associacao',999)<=6),
    'Novo', 'Base Giro'
)

# Blocos texto adicionais
base_princ['bloco_principalidade'] = base_princ.apply(lambda r: f"Faixa: {r.get('faixa_categoria')} | Var pts: {r.get('var_pontos')} | Queda flag: {r.get('queda_flag')} | Soma quedas: {r.get('soma_quedas')}", axis=1)
base_princ['bloco_familias'] = base_princ.apply(lambda r: f"Pontuações -> Acesso: {r.get('score_acesso')} | Fluxo: {r.get('score_fluxo')} | SOW Cartão: {r.get('score_sow')}", axis=1)
base_princ['bloco_tipo_pix'] = base_princ.apply(lambda r: f"Tipo Pessoa: {r.get('tipo_pessoa')} | Chaves PIX fortes: {r.get('qtde_chaves_pix_fortes')}", axis=1)
base_princ['bloco_assoc'] = base_princ['class_assoc'].apply(lambda v: f"Classificação Associação: {v}")

# Bloco ISA (recalcular se necessário)
base_princ['bloco_isa'] = base_princ.apply(lambda r: (
    f"ISA atual: {r.get('isa_media')} | ISA média 3M: {r.get('isa_media_3m_calc'):.2f}" if pd.notnull(r.get('isa_media_3m_calc')) else (
        f"ISA atual: {r.get('isa_media')}" if pd.notnull(r.get('isa_media')) else ''
    )
), axis=1)

base_princ['bloco_prod_basicos'] = base_princ['prod_cat1_faltantes'].apply(lambda lst: ('Produtos básicos a ofertar: ' + ', '.join(lst)) if lst else 'Produtos básicos completos')
base_princ['bloco_fluxo'] = base_princ['prod_cat2_metricas'].apply(lambda m: 'Indicadores fluxo / SOW: ' + '; '.join(m))

# Título e descrição (incluyendo nuevos blocos)
base_princ['titulo'] = base_princ.apply(lambda r: f"Oferta Principalidade - {r.get('cpf_cnpj')}", axis=1)
base_princ['descricao'] = base_princ.apply(lambda r: ' | '.join([
    f"Segmento: {r.get('segmento')}",
    r.get('bloco_tipo_pix',''),
    r.get('bloco_assoc',''),
    r.get('bloco_isa',''),
    r.get('bloco_principalidade',''),
    r.get('bloco_familias',''),
    f"MC Total Atual: {r.get('cash_total')}",
    r.get('bloco_prod_basicos',''),
    r.get('bloco_fluxo','')
]), axis=1)

# Categoria (1 se há produto faltante, senão 2)
base_princ['categoriaChamadoId'] = base_princ['prod_cat1_faltantes'].apply(lambda l: 1 if len(l)>0 else 2)
base_princ['nomeCategoria'] = base_princ['categoriaChamadoId'].map({1:'Acesso',2:'Fluxo de Caixa'})
base_princ['descricaoCategoria'] = base_princ['nomeCategoria'].map({'Acesso':'Ofertar Produtos e Serviços Básicos','Fluxo de Caixa':'Ofertar Movimentações e Uso de Crédito'})

# Distribuir datas
base_princ = base_princ.sort_values('pontos_principalidade').reset_index(drop=True)
qtd = len(base_princ)
ini_mes = datetime.today().replace(day=1).date()
semanas = [0,7,14,21]
base_princ['dataNecessidade'] = base_princ.index.map(lambda i: (ini_mes + timedelta(days=semanas[(i * 4 // (qtd if qtd>0 else 1)) % 4])).isoformat()+'T08:00:00.000Z')

base_princ['categoriaChamado'] = base_princ.apply(lambda r: {
    'categoriaChamadoId': int(r['categoriaChamadoId']),
    'nomeCategoria': r['nomeCategoria'],
    'descricaoCategoria': r['descricaoCategoria'],
    'dataCriacao': datetime.today().date().isoformat(),
    'departamentoId': int(r.get('cod_agencia',0))
}, axis=1)

# Preparar base_final renombrando columnas esperadas
base_final = base_princ.rename(columns={'cod_agencia': 'numeroAgencia', 'nome_agencia': 'nomeDepartamento'})

# Responsable / solicitante
usuario_default = globals().get('user', '') or globals().get('USER', '')
base_final['usuarioResponsavel'] = base_final.get('gestor', usuario_default)
base_final['usuarioSolicitante'] = base_final['usuarioResponsavel']
base_final['statusChamado'] = 'RASCUNHO'
base_final['departamentoId'] = base_final['numeroAgencia']

# Selección columnas de salida
cols_saida = ['titulo', 'usuarioResponsavel', 'usuarioSolicitante', 'numeroAgencia',
             'descricao', 'statusChamado', 'departamentoId', 'nomeDepartamento',
             'dataNecessidade', 'categoriaChamado']

resultado_chamados = base_final.loc[:, [c for c in cols_saida if c in base_final.columns]].copy()

# Salvar arquivos parquet
saida_dir = RUTAS.get('saida_dir_giro') if isinstance(RUTAS, dict) else None
if saida_dir:
    parquet_completo = os.path.join(saida_dir, 'giro_completo_principalidade.parquet')
    parquet_chamados = os.path.join(saida_dir, 'giro_chamados_principalidade.parquet')
    try:
        base_princ.to_parquet(parquet_completo, index=False)
        resultado_chamados.to_parquet(parquet_chamados, index=False)
        print({'completo': parquet_completo, 'chamados': parquet_chamados, 'linhas_chamados': len(resultado_chamados)})
    except Exception as e:
        print('Erro ao salvar parquet:', e)
else:
    print('RUTAS["saida_dir_giro"] não encontrada; não foi possível salvar os arquivos.')

resultado_chamados.to_parquet(rf"")


In [ ]:
# Aplicar filtros e preparar base principal (usa base_princ já carregada do parquet associados_queda_principalidade)
if 'base_princ' not in globals():
    raise RuntimeError('base_princ não carregada. Execute a célula de leitura do parquet base antes desta.')

if base_princ.empty:
    raise ValueError('base_princ está vazia. Verifique o parquet base.')

# Normalizar chave
def normaliza_doc(valor):
    if pd.isna(valor):
        return valor
    v = re.sub(r'\D','', str(valor))
    return v.lstrip('0')

if 'cpf_cnpj' not in base_princ.columns:
    possiveis_chaves = [c for c in base_princ.columns if 'cpf' in c.lower() or 'cnpj' in c.lower() or 'doc' in c.lower()]
    if possiveis_chaves:
        base_princ.rename(columns={possiveis_chaves[0]:'cpf_cnpj'}, inplace=True)
if 'cpf_cnpj' not in base_princ.columns:
    raise ValueError('Coluna cpf_cnpj ausente na base principalidade mês.')
base_princ['cpf_cnpj'] = base_princ['cpf_cnpj'].map(normaliza_doc)

# Inadimplentes
path_inad = RUTAS['inadimplentes']
if os.path.exists(path_inad):
    inad = pd.read_parquet(path_inad)
    for alt in ['cpf','cnpj','documento','doc','num_cpf_cnpj']:
        if alt in inad.columns and 'cpf_cnpj' not in inad.columns:
            inad.rename(columns={alt:'cpf_cnpj'}, inplace=True)
    if 'cpf_cnpj' not in inad.columns:
        inad['cpf_cnpj'] = inad.iloc[:,0]
    inad['cpf_cnpj'] = inad['cpf_cnpj'].map(normaliza_doc)
    inad = inad[['cpf_cnpj']].drop_duplicates()
    inad['flag_inadimplente'] = 1
    base_princ = base_princ.merge(inad, on='cpf_cnpj', how='left')
else:
    base_princ['flag_inadimplente'] = np.nan

# Risco BBM (normalizar caixa / acentuação mínima)
if 'nivel_risco' in base_princ.columns:
    base_princ['nivel_risco_norm'] = base_princ['nivel_risco'].astype(str).str.upper().str.strip()
    riscos_ok = [r.upper() for r in PARAMS['filtros_risco_bbm']]
    base_princ = base_princ[base_princ['nivel_risco_norm'].isin(riscos_ok)]
else:
    print('Aviso: coluna nivel_risco ausente.')

# Sem inadimplência
base_princ = base_princ[(base_princ['flag_inadimplente'].isna()) | (base_princ['flag_inadimplente']!=1)]

# Merge dashboard (inclui ult_movimento)
if 'assoc_dash' in globals():
    assoc_dash['cpf_cnpj'] = assoc_dash['cpf_cnpj'].map(normaliza_doc)
    base_princ = base_princ.merge(assoc_dash, on='cpf_cnpj', how='left')
else:
    print('Dashboard não carregado: execute a célula de carga do dashboard/ISA antes desta.')

# Filtro movimento 20-45 dias usando ult_movimento
if 'ult_movimento' in base_princ.columns:
    hoje = pd.Timestamp.today().normalize()
    base_princ['ult_movimento_dt'] = pd.to_datetime(base_princ['ult_movimento'], errors='coerce')
    base_princ['dias_sem_mov'] = (hoje - base_princ['ult_movimento_dt']).dt.days
    lim_inf, lim_sup = PARAMS['dias_sem_movimentacao']
    base_princ = base_princ[(base_princ['dias_sem_mov']>=lim_inf) & (base_princ['dias_sem_mov']<=lim_sup)]
else:
    print('Coluna ult_movimento ausente no dashboard para filtro de movimento.')

# Merge ISA se disponível
if 'isa_sel' in globals() and isinstance(isa_sel, pd.DataFrame) and not isa_sel.empty and 'cpf_cnpj' in isa_sel.columns:
    isa_sel['cpf_cnpj'] = isa_sel['cpf_cnpj'].map(normaliza_doc)
    if 'gestor' in base_princ.columns and 'gestor' in isa_sel.columns:
        base_princ = base_princ.merge(isa_sel, on=['cpf_cnpj','gestor'], how='left')
    else:
        base_princ = base_princ.merge(isa_sel, on='cpf_cnpj', how='left')
else:
    base_princ['isa_media_3m_calc'] = np.nan

# Corte ISA >= 2 (média atual ou média 3M). Remove quem está fora.
if 'isa_media' in base_princ.columns:
    base_princ = base_princ[(base_princ['isa_media'].fillna(0) >= 2) | (base_princ.get('isa_media_3m_calc', pd.Series(dtype=float)).fillna(0) >= 2)]

print('Base após filtros (inclui corte ISA>=2):', base_princ.shape)
base_princ.head(2)

In [ ]:
# Gerar parquets: completo (dados filtrados) e card Kanban
import json

if 'base_princ' not in globals() or 'resultado_chamados' not in globals():
    raise RuntimeError('Execute as células anteriores antes de gerar os parquets finais.')

# 1. Parquet completo já salvo anteriormente, mas reforçamos aqui se desejar atualizar
parquet_completo = os.path.join(RUTAS['saida_dir_giro'], 'giro_completo_principalidade.parquet')
base_princ.to_parquet(parquet_completo, index=False)

# 2. Parquet apenas com campos necessários para o card (Kanban)
cols_card = [
    'titulo',
    'descricao',
    'categoriaChamado',
    'usuarioResponsavel',
    'usuarioSolicitante',
    'dataNecessidade',
    'statusChamado',
    'numeroAgencia',
    'nomeDepartamento',
    'departamentoId'
]
card_df = resultado_chamados[cols_card].copy()

# Opcional: serializar categoriaChamado em JSON para consumo externo se necessário
card_df['categoriaChamado_json'] = card_df['categoriaChamado'].apply(lambda x: json.dumps(x, ensure_ascii=False))

parquet_card = os.path.join(RUTAS['saida_dir_giro'], 'giro_cards_kanban.parquet')
card_df.to_parquet(parquet_card, index=False)

print({'parquet_completo': parquet_completo, 'parquet_card': parquet_card, 'linhas_card': len(card_df)})
card_df.head(1)

In [ ]:
# PIPELINE FINAL UNIFICADA
import json
from datetime import datetime, timedelta

if 'base_princ' not in globals():
    raise RuntimeError('base_princ ausente. Execute leitura base.')

# Garantir campos essenciais derivados
if 'prod_cat1_faltantes' not in base_princ.columns:
    raise RuntimeError('Execute a célula de derivação de produtos antes da pipeline final.')

# Família pontuação por grupos (Acesso = produtos básicos, Fluxo = cash, SOW = sow_cartao)
base_princ['score_acesso'] = base_princ.get('pontos_produtos_basicos', np.nan)
base_princ['score_fluxo'] = base_princ.get('cash_total_fator', np.nan)
base_princ['score_sow'] = base_princ.get('sow_cartao', np.nan)

# Classificação novo associado (flg_novo_assoc ou meses_desde_associacao)
base_princ['class_assoc'] = np.where(
    (base_princ.get('flg_novo_assoc',0)==1) | (base_princ.get('meses_desde_associacao',999)<=6),
    'Novo', 'Base Giro'
)

# Blocos texto adicionais
base_princ['bloco_principalidade'] = base_princ.apply(lambda r: f"Faixa: {r.get('faixa_categoria')} | Var pts: {r.get('var_pontos')} | Queda flag: {r.get('queda_flag')} | Soma quedas: {r.get('soma_quedas')}", axis=1)
base_princ['bloco_familias'] = base_princ.apply(lambda r: f"Pontuações -> Acesso: {r.get('score_acesso')} | Fluxo: {r.get('score_fluxo')} | SOW Cartão: {r.get('score_sow')}", axis=1)
base_princ['bloco_tipo_pix'] = base_princ.apply(lambda r: f"Tipo Pessoa: {r.get('tipo_pessoa')} | Chaves PIX fortes: {r.get('qtde_chaves_pix_fortes')}", axis=1)
base_princ['bloco_assoc'] = base_princ['class_assoc'].apply(lambda v: f"Classificação Associação: {v}")

# Bloco ISA (recalcular se necessário)
base_princ['bloco_isa'] = base_princ.apply(lambda r: (
    f"ISA atual: {r.get('isa_media')} | ISA média 3M: {r.get('isa_media_3m_calc'):.2f}" if pd.notnull(r.get('isa_media_3m_calc')) else (
        f"ISA atual: {r.get('isa_media')}" if pd.notnull(r.get('isa_media')) else ''
    )
), axis=1)

base_princ['bloco_prod_basicos'] = base_princ['prod_cat1_faltantes'].apply(lambda lst: ('Produtos básicos a ofertar: ' + ', '.join(lst)) if lst else 'Produtos básicos completos')
base_princ['bloco_fluxo'] = base_princ['prod_cat2_metricas'].apply(lambda m: 'Indicadores fluxo / SOW: ' + '; '.join(m))

# Título e descrição (incluindo novos blocos)
base_princ['titulo'] = base_princ.apply(lambda r: f"Oferta Principalidade - {r.get('cpf_cnpj')}", axis=1)
base_princ['descricao'] = base_princ.apply(lambda r: ' | '.join([
    f"Segmento: {r.get('segmento')}",
    r.get('bloco_tipo_pix',''),
    r.get('bloco_assoc',''),
    r.get('bloco_isa',''),
    r.get('bloco_principalidade',''),
    r.get('bloco_familias',''),
    f"MC Total Atual: {r.get('cash_total')}",
    r.get('bloco_prod_basicos',''),
    r.get('bloco_fluxo','')
]), axis=1)

# Categoria (1 se há produto faltante, senão 2)
base_princ['categoriaChamadoId'] = base_princ['prod_cat1_faltantes'].apply(lambda l: 1 if len(l)>0 else 2)
base_princ['nomeCategoria'] = base_princ['categoriaChamadoId'].map({1:'Acesso',2:'Fluxo de Caixa'})
base_princ['descricaoCategoria'] = base_princ['nomeCategoria'].map({'Acesso':'Ofertar Produtos e Serviços Básicos','Fluxo de Caixa':'Ofertar Movimentações e Uso de Crédito'})

# Distribuir datas
base_princ = base_princ.sort_values('pontos_principalidade').reset_index(drop=True)
qtd = len(base_princ)
ini_mes = datetime.today().replace(day=1).date()
semanas = [0,7,14,21]
base_princ['dataNecessidade'] = base_princ.index.map(lambda i: (ini_mes + timedelta(days=semanas[(i * 4 // (qtd if qtd>0 else 1)) % 4])).isoformat()+'T08:00:00.000Z')

base_princ['categoriaChamado'] = base_princ.apply(lambda r: {
    'categoriaChamadoId': int(r['categoriaChamadoId']),
    'nomeCategoria': r['nomeCategoria'],
    'descricaoCategoria': r['descricaoCategoria'],
    'dataCriacao': datetime.today().date().isoformat(),
    'departamentoId': int(r.get('cod_agencia',0))
}, axis=1)

# Preparar base_final renombrando columnas esperadas
base_final = base_princ.rename(columns={'cod_agencia': 'numeroAgencia', 'nome_agencia': 'nomeDepartamento'})

# Responsable / solicitante
usuario_default = globals().get('user', '') or globals().get('USER', '')
base_final['usuarioResponsavel'] = base_final.get('gestor', usuario_default)
base_final['usuarioSolicitante'] = base_final['usuarioResponsavel']
base_final['statusChamado'] = 'RASCUNHO'
base_final['departamentoId'] = base_final['numeroAgencia']

# Selección columnas de salida
cols_saida = ['titulo', 'usuarioResponsavel', 'usuarioSolicitante', 'numeroAgencia',
             'descricao', 'statusChamado', 'departamentoId', 'nomeDepartamento',
             'dataNecessidade', 'categoriaChamado']

resultado_chamados = base_final.loc[:, [c for c in cols_saida if c in base_final.columns]].copy()

# Salvar arquivos parquet
saida_dir = RUTAS.get('saida_dir_giro') if isinstance(RUTAS, dict) else None
if saida_dir:
    parquet_completo = os.path.join(saida_dir, 'giro_completo_principalidade.parquet')
    parquet_chamados = os.path.join(saida_dir, 'giro_chamados_principalidade.parquet')
    try:
        base_princ.to_parquet(parquet_completo, index=False)
        resultado_chamados.to_parquet(parquet_chamados, index=False)
        print({'completo': parquet_completo, 'chamados': parquet_chamados, 'linhas_chamados': len(resultado_chamados)})
    except Exception as e:
        print('Erro ao salvar parquet:', e)
else:
    print('RUTAS["saida_dir_giro"] não encontrada; não foi possível salvar os arquivos.')

resultado_chamados.to_parquet(rf"")


In [ ]:
# Aplicar filtros e preparar base principal (usa base_princ já carregada do parquet associados_queda_principalidade)
if 'base_princ' not in globals():
    raise RuntimeError('base_princ não carregada. Execute a célula de leitura do parquet base antes desta.')

if base_princ.empty:
    raise ValueError('base_princ está vazia. Verifique o parquet base.')

# Normalizar chave
def normaliza_doc(valor):
    if pd.isna(valor):
        return valor
    v = re.sub(r'\D','', str(valor))
    return v.lstrip('0')

if 'cpf_cnpj' not in base_princ.columns:
    possiveis_chaves = [c for c in base_princ.columns if 'cpf' in c.lower() or 'cnpj' in c.lower() or 'doc' in c.lower()]
    if possiveis_chaves:
        base_princ.rename(columns={possiveis_chaves[0]:'cpf_cnpj'}, inplace=True)
if 'cpf_cnpj' not in base_princ.columns:
    raise ValueError('Coluna cpf_cnpj ausente na base principalidade mês.')
base_princ['cpf_cnpj'] = base_princ['cpf_cnpj'].map(normaliza_doc)

# Inadimplentes
path_inad = RUTAS['inadimplentes']
if os.path.exists(path_inad):
    inad = pd.read_parquet(path_inad)
    for alt in ['cpf','cnpj','documento','doc','num_cpf_cnpj']:
        if alt in inad.columns and 'cpf_cnpj' not in inad.columns:
            inad.rename(columns={alt:'cpf_cnpj'}, inplace=True)
    if 'cpf_cnpj' not in inad.columns:
        inad['cpf_cnpj'] = inad.iloc[:,0]
    inad['cpf_cnpj'] = inad['cpf_cnpj'].map(normaliza_doc)
    inad = inad[['cpf_cnpj']].drop_duplicates()
    inad['flag_inadimplente'] = 1
    base_princ = base_princ.merge(inad, on='cpf_cnpj', how='left')
else:
    base_princ['flag_inadimplente'] = np.nan

# Risco BBM (normalizar caixa / acentuação mínima)
if 'nivel_risco' in base_princ.columns:
    base_princ['nivel_risco_norm'] = base_princ['nivel_risco'].astype(str).str.upper().str.strip()
    riscos_ok = [r.upper() for r in PARAMS['filtros_risco_bbm']]
    base_princ = base_princ[base_princ['nivel_risco_norm'].isin(riscos_ok)]
else:
    print('Aviso: coluna nivel_risco ausente.')

# Sem inadimplência
base_princ = base_princ[(base_princ['flag_inadimplente'].isna()) | (base_princ['flag_inadimplente']!=1)]

# Merge dashboard (inclui ult_movimento)
if 'assoc_dash' in globals():
    assoc_dash['cpf_cnpj'] = assoc_dash['cpf_cnpj'].map(normaliza_doc)
    base_princ = base_princ.merge(assoc_dash, on='cpf_cnpj', how='left')
else:
    print('Dashboard não carregado: execute a célula de carga do dashboard/ISA antes desta.')

# Filtro movimento 20-45 dias usando ult_movimento
if 'ult_movimento' in base_princ.columns:
    hoje = pd.Timestamp.today().normalize()
    base_princ['ult_movimento_dt'] = pd.to_datetime(base_princ['ult_movimento'], errors='coerce')
    base_princ['dias_sem_mov'] = (hoje - base_princ['ult_movimento_dt']).dt.days
    lim_inf, lim_sup = PARAMS['dias_sem_movimentacao']
    base_princ = base_princ[(base_princ['dias_sem_mov']>=lim_inf) & (base_princ['dias_sem_mov']<=lim_sup)]
else:
    print('Coluna ult_movimento ausente no dashboard para filtro de movimento.')

# Merge ISA se disponível
if 'isa_sel' in globals() and isinstance(isa_sel, pd.DataFrame) and not isa_sel.empty and 'cpf_cnpj' in isa_sel.columns:
    isa_sel['cpf_cnpj'] = isa_sel['cpf_cnpj'].map(normaliza_doc)
    if 'gestor' in base_princ.columns and 'gestor' in isa_sel.columns:
        base_princ = base_princ.merge(isa_sel, on=['cpf_cnpj','gestor'], how='left')
    else:
        base_princ = base_princ.merge(isa_sel, on='cpf_cnpj', how='left')
else:
    base_princ['isa_media_3m_calc'] = np.nan

# Corte ISA >= 2 (média atual ou média 3M). Remove quem está fora.
if 'isa_media' in base_princ.columns:
    base_princ = base_princ[(base_princ['isa_media'].fillna(0) >= 2) | (base_princ.get('isa_media_3m_calc', pd.Series(dtype=float)).fillna(0) >= 2)]

print('Base após filtros (inclui corte ISA>=2):', base_princ.shape)
base_princ.head(2)

In [ ]:
# Gerar parquets: completo (dados filtrados) e card Kanban
import json

if 'base_princ' not in globals() or 'resultado_chamados' not in globals():
    raise RuntimeError('Execute as células anteriores antes de gerar os parquets finais.')

# 1. Parquet completo já salvo anteriormente, mas reforçamos aqui se desejar atualizar
parquet_completo = os.path.join(RUTAS['saida_dir_giro'], 'giro_completo_principalidade.parquet')
base_princ.to_parquet(parquet_completo, index=False)

# 2. Parquet apenas com campos necessários para o card (Kanban)
cols_card = [
    'titulo',
    'descricao',
    'categoriaChamado',
    'usuarioResponsavel',
    'usuarioSolicitante',
    'dataNecessidade',
    'statusChamado',
    'numeroAgencia',
    'nomeDepartamento',
    'departamentoId'
]
card_df = resultado_chamados[cols_card].copy()

# Opcional: serializar categoriaChamado em JSON para consumo externo se necessário
card_df['categoriaChamado_json'] = card_df['categoriaChamado'].apply(lambda x: json.dumps(x, ensure_ascii=False))

parquet_card = os.path.join(RUTAS['saida_dir_giro'], 'giro_cards_kanban.parquet')
card_df.to_parquet(parquet_card, index=False)

print({'parquet_completo': parquet_completo, 'parquet_card': parquet_card, 'linhas_card': len(card_df)})
card_df.head(1)

In [ ]:
# PIPELINE FINAL UNIFICADA
import json
from datetime import datetime, timedelta

if 'base_princ' not in globals():
    raise RuntimeError('base_princ ausente. Execute leitura base.')

# Garantir campos essenciais derivados
if 'prod_cat1_faltantes' not in base_princ.columns:
    raise RuntimeError('Execute a célula de derivação de produtos antes da pipeline final.')

# Família pontuação por grupos (Acesso = produtos básicos, Fluxo = cash, SOW = sow_cartao)
base_princ['score_acesso'] = base_princ.get('pontos_produtos_basicos', np.nan)
base_princ['score_fluxo'] = base_princ.get('cash_total_fator', np.nan)
base_princ['score_sow'] = base_princ.get('sow_cartao', np.nan)

# Classificação novo associado (flg_novo_assoc ou meses_desde_associacao)
base_princ['class_assoc'] = np.where(
    (base_princ.get('flg_novo_assoc',0)==1) | (base_princ.get('meses_desde_associacao',999)<=6),
    'Novo', 'Base Giro'
)

# Blocos texto adicionais
base_princ['bloco_principalidade'] = base_princ.apply(lambda r: f"Faixa: {r.get('faixa_categoria')} | Var pts: {r.get('var_pontos')} | Queda flag: {r.get('queda_flag')} | Soma quedas: {r.get('soma_quedas')}", axis=1)
base_princ['bloco_familias'] = base_princ.apply(lambda r: f"Pontuações -> Acesso: {r.get('score_acesso')} | Fluxo: {r.get('score_fluxo')} | SOW Cartão: {r.get('score_sow')}", axis=1)
base_princ['bloco_tipo_pix'] = base_princ.apply(lambda r: f"Tipo Pessoa: {r.get('tipo_pessoa')} | Chaves PIX fortes: {r.get('qtde_chaves_pix_fortes')}", axis=1)
base_princ['bloco_assoc'] = base_princ['class_assoc'].apply(lambda v: f"Classificação Associação: {v}")

# Bloco ISA (recalcular se necessário)
base_princ['bloco_isa'] = base_princ.apply(lambda r: (
    f"ISA atual: {r.get('isa_media')} | ISA média 3M: {r.get('isa_media_3m_calc'):.2f}" if pd.notnull(r.get('isa_media_3m_calc')) else (
        f"ISA atual: {r.get('isa_media')}" if pd.notnull(r.get('isa_media')) else ''
    )
), axis=1)

base_princ['bloco_prod_basicos'] = base_princ['prod_cat1_faltantes'].apply(lambda lst: ('Produtos básicos a ofertar: ' + ', '.join(lst)) if lst else 'Produtos básicos completos')
base_princ['bloco_fluxo'] = base_princ['prod_cat2_metricas'].apply(lambda m: 'Indicadores fluxo / SOW: ' + '; '.join(m))

# Título e descrição (incluindo novos blocos)
base_princ['titulo'] = base_princ.apply(lambda r: f"Oferta Principalidade - {r.get('cpf_cnpj')}", axis=1)
base_princ['descricao'] = base_princ.apply(lambda r: ' | '.join([
    f"Segmento: {r.get('segmento')}",
    r.get('bloco_tipo_pix',''),
    r.get('bloco_assoc',''),
    r.get('bloco_isa',''),
    r.get('bloco_principalidade',''),
    r.get('bloco_familias',''),
    f"MC Total Atual: {r.get('cash_total')}",
    r.get('bloco_prod_basicos',''),
    r.get('bloco_fluxo','')
]), axis=1)

# Categoria (1 se há produto faltante, senão 2)
base_princ['categoriaChamadoId'] = base_princ['prod_cat1_faltantes'].apply(lambda l: 1 if len(l)>0 else 2)
base_princ['nomeCategoria'] = base_princ['categoriaChamadoId'].map({1:'Acesso',2:'Fluxo de Caixa'})
base_princ['descricaoCategoria'] = base_princ['nomeCategoria'].map({'Acesso':'Ofertar Produtos e Serviços Básicos','Fluxo de Caixa':'Ofertar Movimentações e Uso de Crédito'})

# Distribuir datas
base_princ = base_princ.sort_values('pontos_principalidade').reset_index(drop=True)
qtd = len(base_princ)
ini_mes = datetime.today().replace(day=1).date()
semanas = [0,7,14,21]
base_princ['dataNecessidade'] = base_princ.index.map(lambda i: (ini_mes + timedelta(days=semanas[(i * 4 // (qtd if qtd>0 else 1)) % 4])).isoformat()+'T08:00:00.000Z')

base_princ['categoriaChamado'] = base_princ.apply(lambda r: {
    'categoriaChamadoId': int(r['categoriaChamadoId']),
    'nomeCategoria': r['nomeCategoria'],
    'descricaoCategoria': r['descricaoCategoria'],
    'dataCriacao': datetime.today().date().isoformat(),
    'departamentoId': int(r.get('cod_agencia',0))
}, axis=1)

# Preparar base_final renombrando columnas esperadas
base_final = base_princ.rename(columns={'cod_agencia': 'numeroAgencia', 'nome_agencia': 'nomeDepartamento'})

# Responsable / solicitante
usuario_default = globals().get('user', '') or globals().get('USER', '')
base_final['usuarioResponsavel'] = base_final.get('gestor', usuario_default)
base_final['usuarioSolicitante'] = base_final['usuarioResponsavel']
base_final['statusChamado'] = 'RASCUNHO'
base_final['departamentoId'] = base_final['numeroAgencia']

# Selección columnas de salida
cols_saida = ['titulo', 'usuarioResponsavel', 'usuarioSolicitante', 'numeroAgencia',
             'descricao', 'statusChamado', 'departamentoId', 'nomeDepartamento',
             'dataNecessidade', 'categoriaChamado']

resultado_chamados = base_final.loc[:, [c for c in cols_saida if c in base_final.columns]].copy()

# Salvar arquivos parquet
saida_dir = RUTAS.get('saida_dir_giro') if isinstance(RUTAS, dict) else None
if saida_dir:
    parquet_completo = os.path.join(saida_dir, 'giro_completo_principalidade.parquet')
    parquet_chamados = os.path.join(saida_dir, 'giro_chamados_principalidade.parquet')
    try:
        base_princ.to_parquet(parquet_completo, index=False)
        resultado_chamados.to_parquet(parquet_chamados, index=False)
        print({'completo': parquet_completo, 'chamados': parquet_chamados, 'linhas_chamados': len(resultado_chamados)})
    except Exception as e:
        print('Erro ao salvar parquet:', e)
else:
    print('RUTAS["saida_dir_giro"] não encontrada; não foi possível salvar os arquivos.')

resultado_chamados.to_parquet(rf"")


In [ ]:
# Aplicar filtros e preparar base principal (usa base_princ já carregada do parquet associados_queda_principalidade)
if 'base_princ' not in globals():
    raise RuntimeError('base_princ não carregada. Execute a célula de leitura do parquet base antes desta.')

if base_princ.empty:
    raise ValueError('base_princ está vazia. Verifique o parquet base.')

# Normalizar chave
def normaliza_doc(valor):
    if pd.isna(valor):
        return valor
    v = re.sub(r'\D','', str(valor))
    return v.lstrip('0')

if 'cpf_cnpj' not in base_princ.columns:
    possiveis_chaves = [c for c in base_princ.columns if 'cpf' in c.lower() or 'cnpj' in c.lower() or 'doc' in c.lower()]
    if possiveis_chaves:
        base_princ.rename(columns={possiveis_chaves[0]:'cpf_cnpj'}, inplace=True)
if 'cpf_cnpj' not in base_princ.columns:
    raise ValueError('Coluna cpf_cnpj ausente na base principalidade mês.')
base_princ['cpf_cnpj'] = base_princ['cpf_cnpj'].map(normaliza_doc)

# Inadimplentes
path_inad = RUTAS['inadimplentes']
if os.path.exists(path_inad):
    inad = pd.read_parquet(path_inad)
    for alt in ['cpf','cnpj','documento','doc','num_cpf_cnpj']:
        if alt in inad.columns and 'cpf_cnpj' not in inad.columns:
            inad.rename(columns={alt:'cpf_cnpj'}, inplace=True)
    if 'cpf_cnpj' not in inad.columns:
        inad['cpf_cnpj'] = inad.iloc[:,0]
    inad['cpf_cnpj'] = inad['cpf_cnpj'].map(normaliza_doc)
    inad = inad[['cpf_cnpj']].drop_duplicates()
    inad['flag_inadimplente'] = 1
    base_princ = base_princ.merge(inad, on='cpf_cnpj', how='left')
else:
    base_princ['flag_inadimplente'] = np.nan

# Risco BBM (normalizar caixa / acentuação mínima)
if 'nivel_risco' in base_princ.columns:
    base_princ['nivel_risco_norm'] = base_princ['nivel_risco'].astype(str).str.upper().str.strip()
    riscos_ok = [r.upper() for r in PARAMS['filtros_risco_bbm']]
    base_princ = base_princ[base_princ['nivel_risco_norm'].isin(riscos_ok)]
else:
    print('Aviso: coluna nivel_risco ausente.')

# Sem inadimplência
base_princ = base_princ[(base_princ['flag_inadimplente'].isna()) | (base_princ['flag_inadimplente']!=1)]

# Merge dashboard (inclui ult_movimento)
if 'assoc_dash' in globals():
    assoc_dash['cpf_cnpj'] = assoc_dash['cpf_cnpj'].map(normaliza_doc)
    base_princ = base_princ.merge(assoc_dash, on='cpf_cnpj', how='left')
else:
    print('Dashboard não carregado: execute a célula de carga do dashboard/ISA antes desta.')

# Filtro movimento 20-45 dias usando ult_movimento
if 'ult_movimento' in base_princ.columns:
    hoje = pd.Timestamp.today().normalize()
    base_princ['ult_movimento_dt'] = pd.to_datetime(base_princ['ult_movimento'], errors='coerce')
    base_princ['dias_sem_mov'] = (hoje - base_princ['ult_movimento_dt']).dt.days
    lim_inf, lim_sup = PARAMS['dias_sem_movimentacao']
    base_princ = base_princ[(base_princ['dias_sem_mov']>=lim_inf) & (base_princ['dias_sem_mov']<=lim_sup)]
else:
    print('Coluna ult_movimento ausente no dashboard para filtro de movimento.')

# Merge ISA se disponível
if 'isa_sel' in globals() and isinstance(isa_sel, pd.DataFrame) and not isa_sel.empty and 'cpf_cnpj' in isa_sel.columns:
    isa_sel['cpf_cnpj'] = isa_sel['cpf_cnpj'].map(normaliza_doc)
    if 'gestor' in base_princ.columns and 'gestor' in isa_sel.columns:
        base_princ = base_princ.merge(isa_sel, on=['cpf_cnpj','gestor'], how='left')
    else:
        base_princ = base_princ.merge(isa_sel, on='cpf_cnpj', how='left')
else:
    base_princ['isa_media_3m_calc'] = np.nan

# Corte ISA >= 2 (média atual ou média 3M). Remove quem está fora.
if 'isa_media' in base_princ.columns:
    base_princ = base_princ[(base_princ['isa_media'].fillna(0) >= 2) | (base_princ.get('isa_media_3m_calc', pd.Series(dtype=float)).fillna(0) >= 2)]

print('Base após filtros (inclui corte ISA>=2):', base_princ.shape)
base_princ.head(2)

In [ ]:
# Gerar parquets: completo (dados filtrados) e card Kanban
import json

if 'base_princ' not in globals() or 'resultado_chamados' not in globals():
    raise RuntimeError('Execute as células anteriores antes de gerar os parquets finais.')

# 1. Parquet completo já salvo anteriormente, mas reforçamos aqui se desejar atualizar
parquet_completo = os.path.join(RUTAS['saida_dir_giro'], 'giro_completo_principalidade.parquet')
base_princ.to_parquet(parquet_completo, index=False)

# 2. Parquet apenas com campos necessários para o card (Kanban)
cols_card = [
    'titulo',
    'descricao',
    'categoriaChamado',
    'usuarioResponsavel',
    'usuarioSolicitante',
    'dataNecessidade',
    'statusChamado',
    'numeroAgencia',
    'nomeDepartamento',
    'departamentoId'
]
card_df = resultado_chamados[cols_card].copy()

# Opcional: serializar categoriaChamado em JSON para consumo externo se necessário
card_df['categoriaChamado_json'] = card_df['categoriaChamado'].apply(lambda x: json.dumps(x, ensure_ascii=False))

parquet_card = os.path.join(RUTAS['saida_dir_giro'], 'giro_cards_kanban.parquet')
card_df.to_parquet(parquet_card, index=False)

print({'parquet_completo': parquet_completo, 'parquet_card': parquet_card, 'linhas_card': len(card_df)})
card_df.head(1)

In [ ]:
# PIPELINE FINAL UNIFICADA
import json
from datetime import datetime, timedelta

if 'base_princ' not in globals():
    raise RuntimeError('base_princ ausente. Execute leitura base.')

# Garantir campos essenciais derivados
if 'prod_cat1_faltantes' not in base_princ.columns:
    raise RuntimeError('Execute a célula de derivação de produtos antes da pipeline final.')

# Família pontuação por grupos (Acesso = produtos básicos, Fluxo = cash, SOW = sow_cartao)
base_princ['score_acesso'] = base_princ.get('pontos_produtos_basicos', np.nan)
base_princ['score_fluxo'] = base_princ.get('cash_total_fator', np.nan)
base_princ['score_sow'] = base_princ.get('sow_cartao', np.nan)

# Classificação novo associado (flg_novo_assoc ou meses_desde_associacao)
base_princ['class_assoc'] = np.where(
    (base_princ.get('flg_novo_assoc',0)==1) | (base_princ.get('meses_desde_associacao',999)<=6),
    'Novo', 'Base Giro'
)

# Blocos texto adicionais
base_princ['bloco_principalidade'] = base_princ.apply(lambda r: f"Faixa: {r.get('faixa_categoria')} | Var pts: {r.get('var_pontos')} | Queda flag: {r.get('queda_flag')} | Soma quedas: {r.get('soma_quedas')}", axis=1)
base_princ['bloco_familias'] = base_princ.apply(lambda r: f"Pontuações -> Acesso: {r.get('score_acesso')} | Fluxo: {r.get('score_fluxo')} | SOW Cartão: {r.get('score_sow')}", axis=1)
base_princ['bloco_tipo_pix'] = base_princ.apply(lambda r: f"Tipo Pessoa: {r.get('tipo_pessoa')} | Chaves PIX fortes: {r.get('qtde_chaves_pix_fortes')}", axis=1)
base_princ['bloco_assoc'] = base_princ['class_assoc'].apply(lambda v: f"Classificação Associação: {v}")

# Bloco ISA (recalcular se necessário)
base_princ['bloco_isa'] = base_princ.apply(lambda r: (
    f"ISA atual: {r.get('isa_media')} | ISA média 3M: {r.get('isa_media_3m_calc'):.2f}" if pd.notnull(r.get('isa_media_3m_calc')) else (
        f"ISA atual: {r.get('isa_media')}" if pd.notnull(r.get('isa_media')) else ''
    )
), axis=1)

base_princ['bloco_prod_basicos'] = base_princ['prod_cat1_faltantes'].apply(lambda lst: ('Produtos básicos a ofertar: ' + ', '.join(lst)) if lst else 'Produtos básicos completos')
base_princ['bloco_fluxo'] = base_princ['prod_cat2_metricas'].apply(lambda m: 'Indicadores fluxo / SOW: ' + '; '.join(m))

# Título e descrição (incluindo novos blocos)
base_princ['titulo'] = base_princ.apply(lambda r: f"Oferta Principalidade - {r.get('cpf_cnpj')}", axis=1)
base_princ['descricao'] = base_princ.apply(lambda r: ' | '.join([
    f"Segmento: {r.get('segmento')}",
    r.get('bloco_tipo_pix',''),
    r.get('bloco_assoc',''),
    r.get('bloco_isa',''),
    r.get('bloco_principalidade',''),
    r.get('bloco_familias',''),
    f"MC Total Atual: {r.get('cash_total')}",
    r.get('bloco_prod_basicos',''),
    r.get('bloco_fluxo','')
]), axis=1)

# Categoria (1 se há produto faltante, senão 2)
base_princ['categoriaChamadoId'] = base_princ['prod_cat1_faltantes'].apply(lambda l: 1 if len(l)>0 else 2)
base_princ['nomeCategoria'] = base_princ['categoriaChamadoId'].map({1:'Acesso',2:'Fluxo de Caixa'})
base_princ['descricaoCategoria'] = base_princ['nomeCategoria'].map({'Acesso':'Ofertar Produtos e Serviços Básicos','Fluxo de Caixa':'Ofertar Movimentações e Uso de Crédito'})

# Distribuir datas
base_princ = base_princ.sort_values('pontos_principalidade').reset_index(drop=True)
qtd = len(base_princ)
ini_mes = datetime.today().replace(day=1).date()
semanas = [0,7,14,21]
base_princ['dataNecessidade'] = base_princ.index.map(lambda i: (ini_mes + timedelta(days=semanas[(i * 4 // (qtd if qtd>0 else 1)) % 4])).isoformat()+'T08:00:00.000Z')

base_princ['categoriaChamado'] = base_princ.apply(lambda r: {
    'categoriaChamadoId': int(r['categoriaChamadoId']),
    'nomeCategoria': r['nomeCategoria'],
    'descricaoCategoria': r['descricaoCategoria'],
    'dataCriacao': datetime.today().date().isoformat(),
    'departamentoId': int(r.get('cod_agencia',0))
}, axis=1)

# Preparar base_final renombrando columnas esperadas
base_final = base_princ.rename(columns={'cod_agencia': 'numeroAgencia', 'nome_agencia': 'nomeDepartamento'})

# Responsable / solicitante
usuario_default = globals().get('user', '') or globals().get('USER', '')
base_final['usuarioResponsavel'] = base_final.get('gestor', usuario_default)
base_final['usuarioSolicitante'] = base_final['usuarioResponsavel']
base_final['statusChamado'] = 'RASCUNHO'
base_final['departamentoId'] = base_final['numeroAgencia']

# Selección columnas de salida
cols_saida = ['titulo', 'usuarioResponsavel', 'usuarioSolicitante', 'numeroAgencia',
             'descricao', 'statusChamado', 'departamentoId', 'nomeDepartamento',
             'dataNecessidade', 'categoriaChamado']

resultado_chamados = base_final.loc[:, [c for c in cols_saida if c in base_final.columns]].copy()

# Salvar arquivos parquet
saida_dir = RUTAS.get('saida_dir_giro') if isinstance(RUTAS, dict) else None
if saida_dir:
    parquet_completo = os.path.join(saida_dir, 'giro_completo_principalidade.parquet')
    parquet_chamados = os.path.join(saida_dir, 'giro_chamados_principalidade.parquet')
    try:
        base_princ.to_parquet(parquet_completo, index=False)
        resultado_chamados.to_parquet(parquet_chamados, index=False)
        print({'completo': parquet_completo, 'chamados': parquet_chamados, 'linhas_chamados': len(resultado_chamados)})
    except Exception as e:
        print('Erro ao salvar parquet:', e)
else:
    print('RUTAS["saida_dir_giro"] não encontrada; não foi possível salvar os arquivos.')

resultado_chamados.to_parquet(rf"")


{  
  "titulo": "string",    
  "usuarioResponsavel": "string", ----> GESTOR
  "usuarioSolicitante": "string", -----> 
  "numeroAgencia": 0, -----> cod_agencia
  "descricao": "string",  --------> descricao texto com detalhes e indices do associado + o texto da oferta de produtos a entregar
  "statusChamado": "RASCUNHO", 
  "departamentoId": 0,
  "nomeDepartamento": "string", --------->  _nom_agencia
  "dataNecessidade": "2025-08-14T11:45:45.991Z",
  "categoriaChamado": {
    "categoriaChamadoId": 0,  ---- > 1 ou 2
    "nomeCategoria": "string", ------> 
    "descricaoCategoria": "string", 
    "dataCriacao": "2025-08-14",
    "departamentoId": 0}
}

Titulo () + nome associado + cpf 

Descrição:
PF OU PJ
ISA atual 
ISA 3M
MC TOTAL ATUAL
MC 6M
INDICES PRINCIPALIDADE

OFERTA : TEXTO conformado por  categoria + descriçao categoria + produto  1 texto : 

  Oferta Acesso  ----->  Categoria 1 
(descriçao categoria) Ofertar Produtos/Serviços Básicos : {} ou [] {add. Produto}-------> Titulo + nome associado + cpf  
. Chave PIX ativa 
· Débito automático
· Cartão de débito
· Cartão de crédito
. Canais digitais
. Credenciamento ativo
· Cobrança
. Folha de Pagamento
. Domicílio
. Open Finance (receptor)

 Oferta Fluxo de Caixa   ----->  Categoria 2 
if % produto in range 
(descriçao categoria) Ofertar Movimentações : [] ou {} {add. Crédito &| cartão}-------> Titulo + nome associado + cpf
. Somatório do total de movimentações:
- Receber (Cash-in)
- Pagar (Cash-out)

Crédito - cartão
. SOW (Share of Wallet)
referente à cartão

Credito - outros
SOW (Share of Wallet)
referente à créditos




In [ ]:
# Celda final: merge inadimplencia + dashboard, salvar parquets (con telefone/gestor), teste CPF 06295052924 y analise resumen
import os, re, json, pandas as pd, numpy as np
from datetime import datetime, date, timedelta

# Normalizar documento
def _norm_doc(v):
    if pd.isna(v): return v
    return re.sub(r'\D','', str(v)).lstrip('0')

# Cargar inadimplentes si existe
inad_path = RUTAS.get('inadimplentes') if isinstance(RUTAS, dict) else None
if inad_path and os.path.exists(inad_path):
    inad = pd.read_parquet(inad_path)
    if 'cpf_cnpj' not in inad.columns:
        for alt in ['cpf','cnpj','documento','doc','num_cpf_cnpj']:
            if alt in inad.columns:
                inad = inad.rename(columns={alt:'cpf_cnpj'})
                break
    inad['cpf_cnpj'] = inad['cpf_cnpj'].map(_norm_doc)
    inad = inad[['cpf_cnpj']].drop_duplicates()
    inad['flag_inadimplente'] = 1
else:
    inad = pd.DataFrame(columns=['cpf_cnpj','flag_inadimplente'])

# Asegurar assoc_dash (telefono y gestor)
dash_path = rf"C:\Users\{USER}\Sicredi\TimeBI_0730 - Documentos\_BASES\arquivos_parquet\associados_totais_tratados_dashboard.parquet"
if os.path.exists(dash_path):
    try:
        assoc_dash = pd.read_parquet(dash_path, columns=['cpf_cnpj','nome_agencia','telefone','gestor','gestor_ldap'])
    except Exception:
        assoc_dash = pd.read_parquet(dash_path, columns=['cpf_cnpj','nome_agencia','telefone','gestor'])
else:
    assoc_dash = pd.DataFrame(columns=['cpf_cnpj','nome_agencia','telefone','gestor','gestor_ldap'])

assoc_dash['cpf_cnpj'] = assoc_dash['cpf_cnpj'].map(_norm_doc)

# Preparar working copy de base_princ
if 'base_princ' not in globals():
    raise RuntimeError("base_princ no cargada.")
bp = base_princ.copy()
if 'cpf_cnpj' in bp.columns:
    bp['cpf_cnpj'] = bp['cpf_cnpj'].map(_norm_doc)

# Merge inadimplencia y assoc_dash
bp = bp.merge(inad, on='cpf_cnpj', how='left')
if not assoc_dash.empty:
    assoc_sel = assoc_dash[['cpf_cnpj','telefone'] + ([c for c in ['gestor_ldap','gestor','nome_agencia'] if c in assoc_dash.columns])]
    bp = bp.merge(assoc_sel.drop_duplicates('cpf_cnpj'), on='cpf_cnpj', how='left')

# asegurar gestor_ldap
if 'gestor_ldap' not in bp.columns:
    bp['gestor_ldap'] = bp.get('gestor', None)

# Filtrar dias sem movimento (20-45) si ult_movimento existe
lim_inf, lim_sup = PARAMS.get('dias_sem_movimentacao', (20,45))
if 'ult_movimento' in bp.columns:
    hoje = pd.Timestamp.today().normalize()
    bp['ult_movimento_dt'] = pd.to_datetime(bp['ult_movimento'], errors='coerce')
    bp['dias_sem_mov'] = (hoje - bp['ult_movimento_dt']).dt.days
    bp = bp[(bp['dias_sem_mov']>=lim_inf) & (bp['dias_sem_mov']<=lim_sup)]
else:
    if 'dias_sem_mov' in bp.columns:
        bp = bp[(bp['dias_sem_mov']>=lim_inf) & (bp['dias_sem_mov']<=lim_sup)]
    else:
        print("Aviso: no hay campo de último movimiento para filtrar.")

# Remover inadimplentes
if 'flag_inadimplente' in bp.columns:
    bp = bp[(bp['flag_inadimplente'].isna()) | (bp['flag_inadimplente']!=1)]

# Guardar parquet completo atómico (incluir telefone y gestor_ldap)
saida_dir = RUTAS.get('saida_dir_giro')
if not saida_dir:
    raise RuntimeError("RUTAS['saida_dir_giro'] no definido.")
full_path = os.path.join(saida_dir, 'giro_completo_principalidade.parquet')
tmp_full = full_path + '.tmp'
# columnas a conservar: intersección de interes
core_cols = ['cpf_cnpj','ano_mes','segmento','cod_agencia','pontos_principalidade','var_pontos','queda_flag','soma_quedas','sow_cartao','cash_total','isa_media','isa_media_3m_calc']
keep_final = [c for c in core_cols if c in bp.columns]
for add in ['telefone','gestor_ldap','descricao_medidas','insight_medidas']:
    if add in bp.columns and add not in keep_final:
        keep_final.append(add)
bp.loc[:, keep_final].to_parquet(tmp_full, index=False)
os.replace(tmp_full, full_path)
print("Guardado parquet completo:", full_path, "| filas:", len(bp))

# Construir resultado_chamados (kanban) con telefone/gestor y salvar
if 'resultado_chamados' in globals() and isinstance(resultado_chamados, pd.DataFrame) and not resultado_chamados.empty:
    kanban_df = resultado_chamados.copy()
else:
    kanban_df = pd.DataFrame({
        'cpf_cnpj': bp['cpf_cnpj'],
        'titulo': bp.get('titulo', bp['cpf_cnpj']),
        'descricao': bp.get('descricao',''),
        'usuarioResponsavel': bp.get('gestor_ldap', None),
        'usuarioSolicitante': bp.get('gestor_ldap', None),
        'numeroAgencia': bp.get('cod_agencia', None),
        'nomeDepartamento': bp.get('nome_agencia', None),
        'dataNecessidade': bp.get('dataNecessidade', None),
        'statusChamado': bp.get('statusChamado', 'RASCUNHO'),
        'departamentoId': bp.get('cod_agencia', None)
    })

if 'cpf_cnpj' in kanban_df.columns and 'cpf_cnpj' in bp.columns:
    mapa = bp[['cpf_cnpj','telefone','gestor_ldap']].drop_duplicates('cpf_cnpj')
    kanban_df = kanban_df.merge(mapa, on='cpf_cnpj', how='left')
else:
    kanban_df['telefone'] = None
    kanban_df['gestor_ldap'] = None

kanban_df['categoriaChamado_json'] = kanban_df.get('categoriaChamado').apply(lambda x: json.dumps(x, ensure_ascii=False) if pd.notnull(x) else None) if 'categoriaChamado' in kanban_df.columns else None

kanban_path = os.path.join(saida_dir, 'giro_cards_kanban.parquet')
tmp_kanban = kanban_path + '.tmp'
kanban_df.to_parquet(tmp_kanban, index=False)
os.replace(tmp_kanban, kanban_path)
print("Guardado Kanban:", kanban_path, "| filas:", len(kanban_df))

# Test CPF: generar markdown para 06295052924
cpf_test = _norm_doc('06295052924')
sel = bp[bp['cpf_cnpj']==cpf_test]
if sel.empty:
    print(f"CPF {cpf_test} no encontrado en la base filtrada. Mostrar primer registro como ejemplo.")
    sel = bp.head(1)
r = sel.iloc[0]

lines = []
lines.append(f"# Ejemplo CPF: {r.get('cpf_cnpj','')}")
lines.append(f"- Nome: {r.get('nom_associado','')}")
lines.append(f"- Gestor (ldap): {r.get('gestor_ldap','')}")
lines.append(f"- Telefones: {r.get('telefone','')}")
lines.append(f"- Segmento: {r.get('segmento','')}")
lines.append(f"- ISA atual: {r.get('isa_media','')} | ISA 3M: {r.get('isa_media_3m_calc','')}")
lines.append(f"- MC Total: {r.get('cash_total','')}")
lines.append("## Indices e variações")
for c in [col for col in bp.columns if col.startswith('delta_') or col.endswith('_flag') or col.startswith('soma_')]:
    lines.append(f"- {c}: {r.get(c)}")
lines.append("## Productos faltantes:")
lines.append(f"- {r.get('prod_cat1_faltantes', [])}")
md_text = "\n".join(lines)
print(md_text)

# Analisis final: conteos y balances
summary = {}
summary['total_filtrados'] = len(bp)
summary['perdieron_puntos_total'] = int(((bp.get('var_pontos',0) < 0)).sum()) if 'var_pontos' in bp.columns else 0
if 'dias_sem_mov' in bp.columns and 'var_pontos' in bp.columns:
    summary['perdieron_puntos_20_45'] = int(((bp['var_pontos']<0) & (bp['dias_sem_mov']>=lim_inf) & (bp['dias_sem_mov']<=lim_sup)).sum())
else:
    summary['perdieron_puntos_20_45'] = None

# ISA balance: columnas isa_YYYYMM o isa_media/isa_media_3m_calc
isa_cols = [c for c in bp.columns if re.match(r'isa_\d{6}$', c)]
if isa_cols:
    last3 = sorted(isa_cols)[-3:]
    bp['isa_3m_mean_calc'] = bp[last3].mean(axis=1)
    summary['isa_3m_media_promedio'] = float(bp['isa_3m_mean_calc'].mean())
else:
    summary['isa_3m_media_promedio'] = float(bp['isa_media_3m_calc'].dropna().mean()) if 'isa_media_3m_calc' in bp.columns else None

# Variaciones de productos (deltas)
delta_cols = [c for c in bp.columns if c.startswith('delta_')]
if delta_cols:
    summary['delta_neg_counts'] = (bp[delta_cols] < 0).sum().to_dict()
else:
    summary['delta_neg_counts'] = {}

# soma_lost_produtos agregado
summary['soma_lost_produtos_total'] = int(bp['soma_lost_produtos'].sum()) if 'soma_lost_produtos' in bp.columns else None

# Guardar resumen
summary_df = pd.DataFrame([summary])
summary_path = os.path.join(saida_dir, 'giro_analise_resumo.parquet')
tmp_sum = summary_path + '.tmp'
summary_df.to_parquet(tmp_sum, index=False)
os.replace(tmp_sum, summary_path)
print("Resumen guardado:", summary_path)
print(summary)